<a href="https://colab.research.google.com/github/Angelaruiga/appweb-2-/blob/main/Gr%C3%A1fico_Network_Publicaciones(1)(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.0 MB/s eta 0:00:00


In [2]:
!pip install matplotlib networkx


In [3]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.9 MB/s eta 0:00:00


In [4]:
!pip install pyngrok

In [5]:
from google.colab import files
import pandas as pd

# Subir archivo desde el sistema local
uploaded = files.upload()

# Asumiendo que sabes el nombre del archivo, por ejemplo 'data_interacciones.csv'
nombre_archivo = next(iter(uploaded))  # Esto obtendrá el nombre del primer archivo subido

# Leer el archivo CSV
df_agrupacion = pd.read_csv(nombre_archivo)

# Mostrar las primeras filas para verificar la carga
print(df_agrupacion)

# Guardar los datos en un archivo CSV para ser utilizado por el script de Streamlit
df_agrupacion.to_csv('data_interacciones.csv', index=False)


Saving data_interacciones.csv to data_interacciones.csv
                                                Grupo1  \
0                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
1                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
2                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
3                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
4                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
..                                                 ...   
155                  IN30_SANCHEZ DEL PINO,MARIA JESUS   
156                  IN30_SANCHEZ DEL PINO,MARIA JESUS   
157  IN35_RODRIGUEZ PIÑERO,MANUEL;CONEJERO GOMEZ,RO...   
158       IN52_RUBIO MARTIN,PATRICIA;LOZANO LOAIZA,EVA   
159       IN52_RUBIO MARTIN,PATRICIA;LOZANO LOAIZA,EVA   

                                                Grupo2  Suma de Interacciones  
0    CO17_AGUILAR DIOSDADO,MANUEL;ARROBA ESPINOSA,A...                      1  
1    CO20_BENAVENTE FERNANDEZ,ISABEL;LUBIAN LOPEZ,S...                      1  
2      

In [6]:
# Guardar el código de Streamlit en un archivo
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# Cargar los datos
data = pd.read_csv('data_interacciones.csv')

# Función para ajustar las etiquetas de los nodos, poner en mayúsculas y reemplazar información tras "_"
def adjust_labels_column(column):
    adjusted_column = column.apply(lambda x: x.split('_')[0].replace(';', '\n').upper() if ';' in x else x.split('_')[0].upper())
    return adjusted_column


# Ajustar las etiquetas de las columnas 'Grupo1' y 'Grupo2'
data['Grupo1'] = adjust_labels_column(data['Grupo1'])
data['Grupo2'] = adjust_labels_column(data['Grupo2'])

# Calcular el tamaño de los nodos basado en el total de interacciones
node_interactions = data.groupby('Grupo1')['Suma de Interacciones'].sum().to_dict()
node_interactions.update(data.groupby('Grupo2')['Suma de Interacciones'].sum().to_dict())

# Crear un widget de lista desplegable para los grupos
unique_groups = list(set(data['Grupo1']).union(set(data['Grupo2'])))
group_selector = st.selectbox('Selecciona un Grupo:', ['All'] + unique_groups)

# Filtrar los datos según el grupo seleccionado
if group_selector != 'All':
    filtered_data = data[(data['Grupo1'] == group_selector) | (data['Grupo2'] == group_selector)]
else:
    filtered_data = data

# Crear el gráfico de red dirigido
G = nx.DiGraph()
for index, row in filtered_data.iterrows():
    G.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Suma de Interacciones'])

pos = nx.spring_layout(G, k=1.0)
labels = {node: node for node in G.nodes()}

# Calcular el tamaño de los nodos basado en el total de interacciones
node_size = [node_interactions.get(node, 1) * 200 for node in G.nodes()]  # Aumentado el tamaño de los nodos

# Configurar el Streamlit
st.title("Publicaciones 2023 en colaboración")

# Crear gráfico con matplotlib y mostrarlo en Streamlit
fig, ax = plt.subplots(figsize=(35, 50))  # Aumentado el tamaño del gráfico
nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='orange', alpha=0.9, ax=ax)
nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.5, edge_color='grey', ax=ax, arrows=True)
nx.draw_networkx_labels(G, pos, labels=labels, font_size=15, font_family="sans-serif", ax=ax)  # Aumentada la fuente de las etiquetas
ax.set_axis_off()
st.pyplot(fig)


Writing app.py


In [7]:
from pyngrok import ngrok

# Autenticar ngrok
ngrok.set_auth_token("2jdVYGyNpE99ZLyZmpR4VITd1OX_6gZQqo3fYAuuFxrSBuaEC")

# Iniciar Streamlit en segundo plano en el puerto 8501
get_ipython().system_raw('streamlit run app.py --server.port 8501 &')

# Exponer el puerto 8501 usando ngrok
public_url = ngrok.connect(addr="8501", bind_tls=True)
print(public_url)

# # Cerrar el túnel ngrok después de un tiempo (opcional)
# import time
# time.sleep(60)  # Mantener el túnel abierto por 60 segundos

# # Cerrar el túnel ngrok
# get_ipython().system_raw(f'ngrok close {public_url}')

ERROR:pyngrok.process.ngrok:t=2024-07-25T06:21:48+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/tunnels/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2024-07-25T06:21:48+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/tunnels/agents\r\n\r\nERR_NGROK_108\r\n.

# Otros intentos


In [ ]:
# from pyngrok import ngrok

# # # Autentica ngrok con tu authtoken
# # ngrok.set_auth_token("vvvv") #cambar por el authoken

In [ ]:
# import pandas as pd
# import networkx as nx
# from bokeh.io import show, output_file
# from bokeh.plotting import figure, from_networkx
# from bokeh.models import Circle, HoverTool, TapTool, BoxSelectTool
# from bokeh.io.export import export_svg
# from bokeh.transform import linear_cmap
# from bokeh.palettes import Spectral4

# # df_agrupacion está definido y tiene los datos correctos
# data = df_agrupacion.copy()

# # Función para ajustar las etiquetas de los nodos, poner en mayúsculas y eliminar información después del guion bajo
# def adjust_labels_column(column):
#     adjusted_column = column.apply(lambda x: x.split('_')[0].upper())
#     return adjusted_column

# # Ajustar las etiquetas de las columnas 'Grupo1' y 'Grupo2'
# data['Grupo1'] = adjust_labels_column(data['Grupo1'])
# data['Grupo2'] = adjust_labels_column(data['Grupo2'])

# # Calcular el tamaño de los nodos basado en el total de interacciones
# node_interactions = data.groupby('Grupo1')['Suma de Interacciones'].sum().to_dict()
# node_interactions.update(data.groupby('Grupo2')['Suma de Interacciones'].sum().to_dict())

# # Crear el gráfico de red dirigido
# G = nx.DiGraph()
# for index, row in data.iterrows():
#     G.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Suma de Interacciones'])

# plot = figure(title="Publicaciones 2023 en colaboración", x_range=(-1.5,1.5), y_range=(-1.5,1.5),
#               tools="pan,wheel_zoom,save,reset", toolbar_location="above")

# # Map node size based on number of interactions
# node_size = {node: node_interactions.get(node, 1) * 100 for node in G.nodes()}
# nx.set_node_attributes(G, node_size, 'size')

# # Create a network graph layout using Bokeh's from_networkx
# network_graph = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

# network_graph.node_renderer.glyph = Circle(size='size', fill_color=Spectral4[0])
# network_graph.edge_renderer.glyph.line_alpha = 0.5
# network_graph.edge_renderer.glyph.line_width = 1.5

# plot.renderers.append(network_graph)

# # Output to static HTML file
# output_file("network_graph.html")
# show(plot)


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import networkx as nx
# import ipywidgets as widgets
# from IPython.display import display

# # Cara de datos
# data = df_agrupacion

# # Crear un widget de lista desplegable para los grupos
# unique_groups = list(set(data['Grupo1']).union(set(data['Grupo2'])))
# group_selector = widgets.Dropdown(
#     options=['All'] + unique_groups,
#     value='All',
#     description='Grupo:',
#     disabled=False
# )

# # Función simplificada para actualizar el gráfico de red agrupada
# def update_clustered_graph(grupo_seleccionado):
#     if grupo_seleccionado == 'All':
#         filtered_data = data_unique
#     else:
#         filtered_data = data_unique[(data_unique['Grupo1'] == grupo_seleccionado) | (data_unique['Grupo2'] == grupo_seleccionado)]

#     # Crear el gráfico de red agrupada
#     G_cluster = nx.Graph()
#     for index, row in filtered_data.iterrows():
#         G_cluster.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Interaccion'])

#     pos = nx.spring_layout(G_cluster, k=0.5)

#     fig, ax = plt.subplots(figsize=(15, 10))
#     nx.draw_networkx(G_cluster, pos, with_labels=True, node_size=500, node_color='skyblue', edge_color='grey', font_size=10, font_family="sans-serif", ax=ax)
#     ax.set_title("Clustered Network Graph (Filtered)", size=15)
#     ax.set_axis_off()
#     plt.show()

# # Crear un interactivo
# interactive_clustered = widgets.interactive(update_clustered_graph, grupo_seleccionado=group_selector)

# # Mostrar el selector y el gráfico interactivo
# display(group_selector)
# display(interactive_clustered)

In [ ]:


# # Pegue o escriba aquí el código de script:
# import matplotlib.pyplot as plt
# import networkx as nx
# import pandas as pd
# import matplotlib.pyplot as plt
# import networkx as nx
# import pandas as pd

# ## Cargar los datos del datas
# data = df_agrupacion

# # Crear el gráfico de red
# G = nx.Graph()

# # Añadir nodos y aristas al gráfico
# for index, row in data.iterrows():
#     G.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Suma de Interacciones'])

# # Calcular el tamaño de los nodos basado en el número de interacciones
# node_size = [G.degree(node) * 800 for node in G.nodes()]
# edge_width = [d['weight'] * 1.5 for (u, v, d) in G.edges(data=True)]

# #
# # Crear el gráfico de red
# G = nx.Graph()

# # Añadir nodos y aristas al gráfico
# for index, row in data.iterrows():
#     G.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Suma de Interacciones'])

# # Calcular el tamaño de los nodos basado en el número de interacciones
# node_size = [G.degree(node) * 1000 for node in G.nodes()]
# edge_width = [d['weight'] * 1.5 for (u, v, d) in G.edges(data=True)]

# # Crear el layout
# pos = nx.spring_layout(G, k=1.5)  # Incrementar k para mayor separación
# # Visualizar la red filtrada
# fig, ax = plt.subplots(figsize=(150, 45))
# nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='orange', alpha=0.9, ax=ax)
# nx.draw_networkx_edges(G ,pos, width=edge_width, alpha=0.5, edge_color='grey', ax=ax)
# nx.draw_networkx_labels(G, pos, font_size=20, font_family='sans-serif', ax=ax) # Completa el argumento font_family

# ax.set_title('Publicaciones entre grupos', fontsize=30)
# plt.show()


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import networkx as nx
# import ipywidgets as widgets
# from IPython.display import display
# import networkx.algorithms.community as nx_comm
# # Simular datos
# data = df_agrupacion


# # Filtrar interacciones únicas
# data_unique = data.groupby(['Grupo1', 'Grupo2']).size().reset_index(name='Interaccion')

# # Crear un widget de lista desplegable para los grupos
# unique_groups = list(set(data['Grupo1']))
# group_selector = widgets.Dropdown(
#     options=['All'] + unique_groups,
#     value='All',
#     description='Grupo:',
#     disabled=False
# )



# # Función para actualizar el gráfico de red agrupada
# def update_clustered_graph(grupo_seleccionado):
#     if grupo_seleccionado == 'All':
#         filtered_data = data_unique
#     else:
#         filtered_data = data_unique[data_unique['Grupo1'] == grupo_seleccionado]

#     # Crear el gráfico de red agrupada
#     G_cluster_unique = nx.Graph()
#     for index, row in filtered_data.iterrows():
#         G_cluster_unique.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Interaccion'])

#     pos_clustered_unique = nx.kamada_kawai_layout(G_cluster_unique)
#     communities_unique = nx_comm.greedy_modularity_communities(G_cluster_unique)

#     color_map_unique = {}
#     for idx, community in enumerate(communities_unique):
#         for name in community:
#             color_map_unique[name] = idx
#     colors_unique = [color_map_unique[node] for node in G_cluster_unique.nodes()]

#     fig, ax = plt.subplots(figsize=(80, 35))
#     nx.draw_networkx_nodes(G_cluster_unique, pos_clustered_unique, node_size=800, node_color=colors_unique, alpha=0.9, ax=ax, cmap=plt.cm.jet)
#     nx.draw_networkx_edges(G_cluster_unique, pos_clustered_unique, width=1.5, alpha=0.5, edge_color='grey', ax=ax)
#     nx.draw_networkx_labels(G_cluster_unique, pos_clustered_unique, font_size=15, font_family="sans-serif", ax=ax)
#     ax.set_title("Clustered Network Graph (Unique Interactions)", size=25)
#     ax.set_axis_off()
#     plt.show()

# # Crear interactivos
# interactive_clustered = widgets.interactive(update_clustered_graph, grupo_seleccionado=group_selector)

# # Mostrar el selector y gráficos interactivos
# display(group_selector)
# display(interactive_clustered)

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import networkx as nx
# import ipywidgets as widgets
# from IPython.display import display

# ## Filtrar interacciones únicas
# data_unique = data.groupby(['Grupo1', 'Grupo2']).size().reset_index(name='Interaccion')

# # Crear un widget de lista desplegable para los grupos
# unique_groups = list(set(data['Grupo1']).union(set(data['Grupo2'])))
# group_selector = widgets.Dropdown(
#     options=['All'] + unique_groups,
#     value='All',
#     description='Grupo:',
#     disabled=False
# )

# # Función para actualizar el gráfico de red agrupada
# def update_clustered_graph(grupo_seleccionado):
#     if grupo_seleccionado == 'All':
#         filtered_data = data_unique
#     else:
#         filtered_data = data_unique[(data_unique['Grupo1'] == grupo_seleccionado) | (data_unique['Grupo2'] == grupo_seleccionado)]

#     # Crear el gráfico de red agrupada
#     G_cluster = nx.Graph()
#     for index, row in filtered_data.iterrows():
#         G_cluster.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Interaccion'])

#     pos = nx.spring_layout(G_cluster, k=0.5)
#     communities = nx.algorithms.community.greedy_modularity_communities(G_cluster)

#     color_map = {}
#     for idx, community in enumerate(communities):
#         for name in community:
#             color_map[name] = idx
#     colors = [color_map[node] for node in G_cluster.nodes()]

#     fig, ax = plt.subplots(figsize=(15, 10))
#     nx.draw_networkx_nodes(G_cluster, pos, node_size=500, node_color=colors, alpha=0.9, ax=ax, cmap=plt.cm.jet)
#     nx.draw_networkx_edges(G_cluster, pos, width=1.5, alpha=0.5, edge_color='grey', ax=ax)
#     nx.draw_networkx_labels(G_cluster, pos, font_size=10, font_family="sans-serif", ax=ax)
#     ax.set_title("Clustered Network Graph (Filtered)", size=15)
#     ax.set_axis_off()
#     plt.show()

# # Crear interactivos
# interactive_clustered = widgets.interactive(update_clustered_graph, grupo_seleccionado=group_selector)

# # Mostrar el selector y gráficos interactivos
# display(group_selector)
# display(interactive_clustered)


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import networkx as nx
# import ipywidgets as widgets
# from IPython.display import display

# # Cara de datos
# data = df_agrupacion

# # Función para ajustar las etiquetas de los nodos, poner en mayúsculas y eliminar información después del guion bajo
# def adjust_labels_column(column):
#     adjusted_column = column.apply(lambda x: x.split('_')[0].upper())
#     return adjusted_column

# # Calcular el tamaño de los nodos basado en el total de interacciones
# node_interactions = data.groupby('Grupo1')['Suma de Interacciones'].sum().to_dict()
# node_interactions.update(data.groupby('Grupo2')['Suma de Interacciones'].sum().to_dict())

# # Crear un widget de lista desplegable para los grupos
# unique_groups = list(set(data['Grupo1']).union(set(data['Grupo2'])))
# group_selector = widgets.Dropdown(
#     options=['All'] + unique_groups,
#     value='All',
#     description='Grupo:',
#     disabled=False
# )

# # Función para actualizar el gráfico de red dirigido
# def update_directed_graph(grupo_seleccionado):
#     if grupo_seleccionado == 'All':
#         filtered_data = data
#     else:
#         filtered_data = data[(data['Grupo1'] == grupo_seleccionado) | (data['Grupo2'] == grupo_seleccionado)]

#     # Crear el gráfico de red dirigido
#     G = nx.DiGraph()
#     for index, row in filtered_data.iterrows():
#         G.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Suma de Interacciones'])

#     pos = nx.spring_layout(G, k=1.0)
#     labels = {node: node for node in G.nodes()}

#     # Calcular el tamaño de los nodos basado en el total de interacciones
#     node_size = [node_interactions.get(node, 1) * 100 for node in G.nodes()]

#     fig, ax = plt.subplots(figsize=(45, 20))
#     nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='orange', alpha=0.9, ax=ax)
#     nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.5, edge_color='grey', ax=ax, arrows=True)
#     nx.draw_networkx_labels(G, pos, labels=labels, font_size=10, font_family="sans-serif", ax=ax)
#     ax.set_title("Publicaciones 2023 en colaboración", size=15)
#     ax.set_axis_off()
#     plt.show()

# # Crear interactivo
# interactive_directed = widgets.interactive(update_directed_graph, grupo_seleccionado=group_selector)

# # Mostrar el selector y gráfico interactivo
# display(group_selector)
# display(interactive_directed)

In [ ]:
# # Guardar el código de Streamlit en un archivo
# %%writefile app.py
# import streamlit as st
# import pandas as pd
# import matplotlib.pyplot as plt
# import networkx as nx

# # Cargar los datos
# data = pd.read_csv('data_interacciones.csv')

# # Función para ajustar las etiquetas de los nodos y poner en mayúsculas
# def adjust_labels_column(column):
#     adjusted_column = column.apply(lambda x: x.replace(';', '\n').upper() if ';' in x else x.upper())
#     return adjusted_column

# # Ajustar las etiquetas de las columnas 'Grupo1' y 'Grupo2'
# data['Grupo1'] = adjust_labels_column(data['Grupo1'])
# data['Grupo2'] = adjust_labels_column(data['Grupo2'])

# # Calcular el tamaño de los nodos basado en el total de interacciones
# node_interactions = data.groupby('Grupo1')['Suma de Interacciones'].sum().to_dict()
# node_interactions.update(data.groupby('Grupo2')['Suma de Interacciones'].sum().to_dict())

# # Crear el gráfico de red dirigido
# G = nx.DiGraph()
# for index, row in data.iterrows():
#     G.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Suma de Interacciones'])

# pos = nx.spring_layout(G, k=1.0)
# labels = {node: node for node in G.nodes()}

# # Calcular el tamaño de los nodos basado en el total de interacciones
# node_size = [node_interactions.get(node, 1) * 100 for node in G.nodes()]

# # Configurar el Streamlit
# st.title("Publicaciones 2023 en colaboración")

# # Crear gráfico con matplotlib y mostrarlo en Streamlit
# fig, ax = plt.subplots(figsize=(35, 20))
# nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='orange', alpha=0.9, ax=ax)
# nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.5, edge_color='grey', ax=ax, arrows=True)
# nx.draw_networkx_labels(G, pos, labels=labels, font_size=10, font_family="sans-serif", ax=ax)
# ax.set_axis_off()
# st.pyplot(fig)


Overwriting app.py


In [ ]:
# from pyngrok import ngrok

# # Autentica ngrok con tu authtoken
# ngrok.set_auth_token("2jdVYGyNpE99ZLyZmpR4VITd1OX_6gZQqo3fYAuuFxrSBuaEC")
